In [1]:
import atexit
import logging
import json
import sys
import time

In [4]:
from kafka import KafkaProducer
from kafka.errors import KafkaError, KafkaTimeoutError
from pyspark import SparkContext
from pyspark.sql.functions import *
from pyspark.streaming import StreamingContext

In [5]:
logger_format = '%(asctime)-15s %(message)s'
logging.basicConfig(format=logger_format)
logger = logging.getLogger('stream-processing')
logger.setLevel(logging.INFO)

In [6]:
topic = 'stockTopic'
target_topic = "transformedStockTopic"
brokers = '127.0.0.1:9092'

In [7]:
def on_shutdown(producer=""):
  '''
  shutdown hook to be called before the shutdown
  :param producer: instance of a kafka producer
  :return: None
  '''
  try:
    logger.info('Flushing pending messages to kafka, timeout is set to 10s')
    # producer.flush(10) 
    logger.info('Finish flushing pending messages to kafka')
  finally:
      try:
        logger.info('Closing the Kafka connection')
        # producer.close(10)
      except Exception as e:
        log_exception('Failed to close kafka connection',e)

In [8]:
def process_stream(stream):

    def send_to_kafka(rdd):
        results = rdd.collect()
        for r in results:
            data = json.dumps(
                {
                    'symbol': r[0],
                    'timestamp': time.time(),
                    'average': r[1]
                }
            )
            try:""
                logger.info('Sending average price %s to kafka' % data)
                kafka_producer.send(target_topic, value=data)
            except KafkaError as error:
                logger.warn('Failed to send average stock price to kafka, caused by: %s', error.message)

    def pair(data):
        record = json.loads(data[1].decode('utf-8'))[0]
        return record.get('StockSymbol'), (float(record.get('LastTradePrice')), 1)

    stream.map(pair).reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1])).map(lambda (k, v): (k, v[0]/v[1])).foreachRDD(send_to_kafka)

IndentationError: unexpected indent (<ipython-input-8-752552aa17c6>, line 14)